<a href="https://colab.research.google.com/github/Mofek/Mofek/blob/main/KG_POC_01_09.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import userdata
userdata.get('secretName')

In [1]:
!pip install --upgrade --quiet  langchain langchain-community langchain-openai langchain-ollama langchain-experimental neo4j tiktoken yfiles_jupyter_graphs python-dotenv
!pip install json-repair
!pip install --upgrade --quiet  langchain-openai langchain-community
!pip install  py2neo
!pip install neo4j
!pip install --upgrade --quiet  langchain-openai langchain-community



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 19.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 60.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.0/52.0 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 204.3/204.3 kB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 20.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 44.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.6/15.6 MB 73.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 396.2/396.2 kB 25.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.7/150.7 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.7/365.7 kB 23.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76

In [11]:
from langchain_core.runnables import  RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.output_parsers import StrOutputParser
import os
from langchain_community.graphs import Neo4jGraph
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import ChatOpenAI
from langchain_community.chat_models import ChatOllama
from langchain_experimental.graph_transformers import LLMGraphTransformer
from neo4j import GraphDatabase
from yfiles_jupyter_graphs import GraphWidget
from langchain_community.vectorstores import Neo4jVector
from langchain_openai import OpenAIEmbeddings
from langchain_community.document_loaders import TextLoader


from langchain_core.documents import Document

from langchain_text_splitters import CharacterTextSplitter
from langchain_community.vectorstores.neo4j_vector import remove_lucene_chars
from google.colab import output
output.enable_custom_widget_manager()
import yaml

from dotenv import load_dotenv

load_dotenv()


True

In [5]:
graph = Neo4jGraph()

In [6]:
loader = TextLoader(file_path="/content/dummytext.txt")
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=250, chunk_overlap=24)
documents = text_splitter.split_documents(documents=docs)
print(documents[:10])

[Document(metadata={'source': '/content/dummytext.txt'}, page_content='1. The Story of Amico’s Family: A Legacy of Love and Tradition'), Document(metadata={'source': '/content/dummytext.txt'}, page_content="In the idyllic village of Santa Caterina, amidst the rolling hills and sun-kissed landscapes of Sicily, lies the genesis of the Caruso family, a lineage intertwined with the island's rich culinary tapestry. The Carusos were not mere inhabitants of"), Document(metadata={'source': '/content/dummytext.txt'}, page_content='not mere inhabitants of the land; they were the keepers of a culinary heritage that spanned generations. Each family member contributed their unique flair, crafting a narrative of flavors that reflected their diverse experiences and deep-seated love'), Document(metadata={'source': '/content/dummytext.txt'}, page_content='and deep-seated love for food.'), Document(metadata={'source': '/content/dummytext.txt'}, page_content='Giovanni Caruso and Maria: The Founding Gener

In [7]:
llm_type = os.getenv("LLM_TYPE", "GPT_MODEL")
llm = ChatOpenAI(temperature=0, model="gpt-4o-mini")
llm_transformer = LLMGraphTransformer(llm=llm)

graph_documents = llm_transformer.convert_to_graph_documents(documents)


In [8]:
graph_documents[0]

GraphDocument(nodes=[Node(id='Amico’S Family', type='Family'), Node(id='Love', type='Concept'), Node(id='Tradition', type='Concept')], relationships=[Relationship(source=Node(id='Amico’S Family', type='Family'), target=Node(id='Love', type='Concept'), type='HAS_LEGACY'), Relationship(source=Node(id='Amico’S Family', type='Family'), target=Node(id='Tradition', type='Concept'), type='HAS_LEGACY')], source=Document(metadata={'source': '/content/dummytext.txt'}, page_content='1. The Story of Amico’s Family: A Legacy of Love and Tradition'))

In [9]:
graph.add_graph_documents(
    graph_documents,
    baseEntityLabel=True,
    include_source=True
)

In [12]:
def showGraph():
    driver = GraphDatabase.driver(
        uri = os.environ["NEO4J_URI"],
        auth = (os.environ["NEO4J_USERNAME"],
                os.environ["NEO4J_PASSWORD"]))
    session = driver.session()
    widget = GraphWidget(graph = session.run("MATCH (s)-[r:!MENTIONS]->(t) RETURN s,r,t").graph())
    widget.node_label_mapping = 'id'
    return widget

showGraph()

GraphWidget(layout=Layout(height='800px', width='100%'))

In [16]:
vector_index = Neo4jVector.from_existing_graph(
    OpenAIEmbeddings(model='text-embedding-3-large',dimensions=384),
    search_type="hybrid",
    node_label="Document",
    text_node_properties=["text"],
    embedding_node_property="embedding"
)
vector_retriever = vector_index.as_retriever()

In [17]:

class Entities(BaseModel):
    """Identifying information about entities."""

    names: list[str] = Field(
        ...,
        description="All the person, organization, or business entities that "
        "appear in the text",
    )

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are extracting organization and person entities from the text.",
        ),
        (
            "human",
            "Use the given format to extract information from the following "
            "input: {question}",
        ),
    ]
)

entity_chain = prompt | llm.with_structured_output(Entities)

In [18]:
entity_chain.invoke({"question": "Who are Nonna Lucia and Giovanni Caruso?"}).names

['Nonna Lucia', 'Giovanni Caruso']

In [21]:
from neo4j import  Driver
from google.colab import userdata

def initialize_fulltext_index(driver: Driver):
    with driver.session() as session:
        # Check if the index exists
        session.run("CREATE FULLTEXT INDEX entity IF NOT EXISTS FOR (n:Document) ON EACH [n.text]")
        # logger.info("Fulltext index 'entity' created or already exists")

# Initialize Neo4j driver
neo4j_driver = GraphDatabase.driver(userdata.get('NEO4J_URI'), auth=("neo4j",userdata.get('NEO4J_PASSWORD')))

# Initialize fulltext index
initialize_fulltext_index(neo4j_driver)

In [24]:
def generate_full_text_query(input: str) -> str:
    words = [el for el in remove_lucene_chars(input).split() if el]
    if not words:
        return ""
    full_text_query = " AND ".join([f"{word}~2" for word in words])
    print(f"Generated Query: {full_text_query}")
    print(full_text_query)
    return full_text_query.strip()


# Fulltext index query
def graph_retriever(question: str) -> str:
    """
    Collects the neighborhood of entities mentioned
    in the question
    """

    print(question)

    result = ""
    entities = entity_chain.invoke({"question": question})
    print(entities)
    for entity in entities.names:
        response = graph.query(
            """CALL db.index.fulltext.queryNodes('entity', $query, {limit:2})
            YIELD node,score
            CALL {
              WITH node
              MATCH (node)-[r:!MENTIONS]->(neighbor)
              RETURN node.id + ' - ' + type(r) + ' -> ' + neighbor.id AS output
              UNION ALL
              WITH node
              MATCH (node)<-[r:!MENTIONS]-(neighbor)
              RETURN neighbor.id + ' - ' + type(r) + ' -> ' +  node.id AS output
            }
            RETURN output LIMIT 50
            """,
            {"query": generate_full_text_query(entity)},
        )
        result += "\n".join([el['output'] for el in response])
    return result

In [25]:
print(graph_retriever("Who is Nonna Lucia?"))

Who is Nonna Lucia?
names=['Nonna Lucia']
Generated Query: Nonna~2 AND Lucia~2
Nonna~2 AND Lucia~2


In [26]:
def full_retriever(question: str):
    graph_data = graph_retriever(question)
    vector_data = [el.page_content for el in vector_retriever.invoke(question)]
    final_data = f"""Graph data:
{graph_data}
vector data:
{"#Document ". join(vector_data)}
    """
    return final_data

In [27]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
Use natural language and be concise.
Answer:"""
prompt = ChatPromptTemplate.from_template(template)

chain = (
        {
            "context": full_retriever,
            "question": RunnablePassthrough(),
        }
    | prompt
    | llm
    | StrOutputParser()
)

In [28]:
chain.invoke(input="Who is Nonna Lucia? Did she teach anyone about restaurants or cooking?")

Who is Nonna Lucia? Did she teach anyone about restaurants or cooking?
names=['Nonna Lucia']
Generated Query: Nonna~2 AND Lucia~2
Nonna~2 AND Lucia~2


'Nonna Lucia is a character known for teaching her family about cooking and restaurants, sharing her culinary knowledge and traditions.'